In [6]:
#!pip install pandas_profiling

In [ ]:
# Imports
import warnings

warnings.simplefilter(action='ignore')

from collections import Counter
from copy import copy, deepcopy
#from gplearn.genetic import SymbolicTransformer
import pandas as pd
import numpy as np
from tqdm import tnrange, tqdm_notebook, tqdm
#from dataset import Dataset
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns; sns.set()
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, cross_val_score, train_test_split, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, recall_score, matthews_corrcoef, precision_score, classification_report, confusion_matrix
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from matplotlib import pyplot
from sklearn.utils.validation import column_or_1d
from sklearn.decomposition import PCA
from datetime import datetime
import pandas_profiling

# Preparing the data

In [55]:
# Importing the data
CSD2016 = pd.read_csv('/Users/apkiener/Desktop/DataThon/CDS_2016_va', encoding = 'latin-1')
CSD2017 = pd.read_csv('/Users/apkiener/Desktop/DataThon/CDS_2017_va', encoding = 'latin-1')
CSD2018 = pd.read_csv('/Users/apkiener/Desktop/DataThon/CDS_2018_va', encoding = 'latin-1')
CSD2019 = pd.read_csv('/Users/apkiener/Desktop/DataThon/CDS_2019_NO_LABEL', encoding = 'latin-1')
clientes = pd.read_csv('/Users/apkiener/Desktop/DataThon/Clientes.csv')
CSD2016

,Id_Cliente,Año Natural,Curso,Asignatura,Tipo Material Educativo,Grupo Editorial,Lengua,Tipo Soporte Actual,Variable 1,Variable 2
0,113958,2016,27,65,3,31,26,1,16.0,70.72
1,113958,2016,27,53,2,31,26,1,16.0,288.48
2,113958,2016,24,79,1,90,26,1,8.0,101.52
3,113958,2016,26,53,3,31,26,1,36.0,199.08
4,113958,2016,28,48,1,90,13,1,22.0,401.06
...,...,...,...,...,...,...,...,...,...,...
612722,210778,2016,37,61,1,90,13,1,51.0,1873.74
612723,210778,2016,37,50,1,90,13,1,5.0,183.70
612724,210778,2016,37,7,1,1,13,1,0.0,0.00
612725,210778,2016,36,42,1,90,13,1,3.0,103.50


In [ ]:
# Renaming columns for all dataframes
CSD2016.rename(columns={'Año Natural': 'Año_Natural', 'Tipo Material Educativo': 'Tipo_Material_Educativo', 'Grupo Editorial': 'Grupo_Editorial', 'Tipo Soporte Actual': 'Tipo_Soporte_Actual', 'Variable 1': 'Variable_1', 'Variable 2': 'Variable_2'}, inplace=True)
CSD2019.rename(columns={'Año natural': 'Año_Natural', 'Tipo Material Educativo': 'Tipo_Material_Educativo', 'Grupo Editorial': 'Grupo_Editorial', 'Tipo Soporte Actual': 'Tipo_Soporte_Actual', 'Variable1': 'Variable_1', 'Variable2': 'Variable_2'}, inplace=True)
CSD2017.columns = CSD2016.columns
CSD2018.columns = CSD2016.columns
clientes.rename(columns={'Comunidad Autónoma': 'Comunidad_Autónoma'}, inplace=True)

In [ ]:
# Cleaning variables from 2018 dataset
CSD2018["Curso"] = CSD2018["Curso"].str.replace("c","")
CSD2018 = CSD2018.replace({"Año_Natural": 18}, {"Año_Natural": 2018}, regex=True)
CSD2018["Curso"] = CSD2018["Curso"].astype('int64')
CSD2019["Variable_2"] = CSD2019["Variable_2"] * 100

In [ ]:
# Creating a unique identifier for each record
CSD2016["Unique_Id"] = (CSD2016["Id_Cliente"].astype(str) + CSD2016["Curso"].astype(str) + CSD2016["Asignatura"].astype(str) + CSD2016["Tipo_Material_Educativo"].astype(str) + CSD2016["Lengua"].astype(str) + CSD2016["Tipo_Soporte_Actual"].astype(str))
CSD2017["Unique_Id"] = (CSD2017["Id_Cliente"].astype(str) + CSD2017["Curso"].astype(str) + CSD2017["Asignatura"].astype(str) + CSD2017["Tipo_Material_Educativo"].astype(str) + CSD2017["Lengua"].astype(str) + CSD2017["Tipo_Soporte_Actual"].astype(str))
CSD2018["Unique_Id"] = (CSD2018["Id_Cliente"].astype(str) + CSD2018["Curso"].astype(str) + CSD2018["Asignatura"].astype(str) + CSD2018["Tipo_Material_Educativo"].astype(str) + CSD2018["Lengua"].astype(str) + CSD2018["Tipo_Soporte_Actual"].astype(str))
CSD2019["Unique_Id"] = (CSD2019["Id_Cliente"].astype(str) + CSD2019["Curso"].astype(str) + CSD2019["Asignatura"].astype(str) + CSD2019["Tipo_Material_Educativo"].astype(str) + CSD2019["Lengua"].astype(str) + CSD2019["Tipo_Soporte_Actual"].astype(str))

In [ ]:
# Checking duplicates for 2016
CSD2016.duplicated().sum()

In [ ]:
# Checking duplicates for 2017
CSD2017.duplicated().sum()

In [ ]:
# Checking duplicates for 2018
CSD2018.duplicated().sum()

In [ ]:
# Checking duplicates for 2019
CSD2019.duplicated().sum()

In [ ]:
# Dropping duplicates
CSD2017.sort_values("Unique_Id", inplace=True)
CSD2017 = CSD2017.drop_duplicates()

In [ ]:
# Changing Grupo_Editiorial as string
CSD2016["Grupo_Editorial"] = CSD2016["Grupo_Editorial"].astype("str")
CSD2017["Grupo_Editorial"] = CSD2017["Grupo_Editorial"].astype("str")
CSD2018["Grupo_Editorial"] = CSD2018["Grupo_Editorial"].astype("str")

In [ ]:
#Defining a function to return the number of years
def courseyears(s):
    if (pd.isnull(s["Años_Curso_PY"]) == True):
        return 1
    else:
        return s["Años_Curso_PY"] + 1

In [ ]:
# Adding column "Groupo_Editorial" from the previous Year and the Number of year of the course
CSD2016["Años_Curso"] = 1

CSD2016_1 = CSD2016[["Unique_Id","Grupo_Editorial","Años_Curso"]]
CSD2016_1.columns = ["Unique_Id","Grupo_Editorial_PY","Años_Curso_PY"]
CSD2017 = pd.merge(CSD2017, CSD2016_1, how = "left", on='Unique_Id')
CSD2017["Años_Curso"] = CSD2017.apply(courseyears, axis=1)
CSD2017 = CSD2017.drop(["Años_Curso_PY"], axis = 1)

CSD2017_1 = CSD2017[["Unique_Id","Grupo_Editorial","Años_Curso"]]
CSD2017_1.columns = ["Unique_Id","Grupo_Editorial_PY","Años_Curso_PY"]
CSD2018 = pd.merge(CSD2018, CSD2017_1,how = "left", on='Unique_Id')
CSD2018["Años_Curso"] = CSD2018.apply(courseyears, axis=1)
CSD2018 = CSD2018.drop(["Años_Curso_PY"], axis = 1)

CSD2018_1 = CSD2018[["Unique_Id","Grupo_Editorial","Años_Curso"]]
CSD2018_1.columns = ["Unique_Id","Grupo_Editorial_PY","Años_Curso_PY"]
CSD2019 = pd.merge(CSD2019, CSD2018_1, how = "left", on='Unique_Id')
CSD2019["Años_Curso"] = CSD2019.apply(courseyears, axis=1)
CSD2019 = CSD2019.drop(["Años_Curso_PY"], axis = 1)

print("Number of courses in 2016: " + CSD2016["Unique_Id"].count().astype(str))
print("Number of courses in 2017: " + CSD2017["Unique_Id"].count().astype(str))
print("Number of courses in 2018: " + CSD2018["Unique_Id"].count().astype(str))
print("Number of courses in 2019: " + CSD2019["Unique_Id"].count().astype(str))

In [ ]:
# Concatenating the dataframes
AllCDS = pd.concat([CSD2016, CSD2017, CSD2018], axis = 0, join = 'outer', ignore_index = False)
print("Number of courses in All Years: " + AllCDS["Unique_Id"].count().astype(str))

In [ ]:
# Defining function to return the type of change from one year to another 
def changes(df):
    if (pd.isnull(df["Grupo_Editorial_PY"]) == True) and (df["Grupo_Editorial"] == "1") :
        return "New_course_SM"
    elif (pd.isnull(df["Grupo_Editorial_PY"]) == True) and (df["Grupo_Editorial"] == "90") :
        return "New_course_No-Use"
    elif (pd.isnull(df["Grupo_Editorial_PY"]) == True) and (df["Grupo_Editorial"] != "1") and (df["Grupo_Editorial"] != "90"):
        return "New_course_Editorial"
    elif (df["Grupo_Editorial_PY"] == "1" ) and (df["Grupo_Editorial"] == "1" ):
        return "SM_to_SM"
    elif (df["Grupo_Editorial_PY"] != "1" ) and (df["Grupo_Editorial_PY"] != "90" ) and (df["Grupo_Editorial"] != "1") and (df["Grupo_Editorial"] != "90"):
        return "Editorial_to_Editorial"
    elif (df["Grupo_Editorial_PY"] == "90" ) and (df["Grupo_Editorial"] == "90"):
        return "No-Use_to_No-Use"
    elif (df["Grupo_Editorial_PY"] == "1" ) and (df["Grupo_Editorial"] != "1" ) and (df["Grupo_Editorial"] != "90" ):
        return "SM_to_Editorial"
    elif (df["Grupo_Editorial_PY"] != "1" ) and (df["Grupo_Editorial"] == "1" ) and (df["Grupo_Editorial"] != "90" ):
        return "Editorial_to_SM"
    elif (df["Grupo_Editorial_PY"] == "90" ) and (df["Grupo_Editorial"] == "1"):
        return "No-Use_to_SM"
    elif (df["Grupo_Editorial_PY"] == "90" ) and (df["Grupo_Editorial"] != "1") and (df["Grupo_Editorial"] != "90"):
        return "No-Use_to_Editorial"
    elif (df["Grupo_Editorial_PY"] == "1" ) and (df["Grupo_Editorial"] == "90"):
        return "SM_to_No-Use"
    elif (df["Grupo_Editorial_PY"] != "1" ) and (df["Grupo_Editorial_PY"] != "90" ) and (df["Grupo_Editorial"] == "90"):
        return "Editorial_to_No-Use"
    else:
        return "FAIL"

In [ ]:
# Creating a new column applying the function already created
AllCDS["Change"] = AllCDS.apply(changes, axis=1)

In [ ]:
# Listing the changes from one year to another by type
AllCDS["Change"].value_counts()

In [ ]:
# Defining a function to return the name of the "Grupo_Editorial"
def grupoeditorialnames(df):
    if (pd.isnull(df["Grupo_Editorial_PY"]) == True):
        return "New_Course"
    elif (df["Grupo_Editorial_PY"] == "1"):
        return "SM"
    elif (df["Grupo_Editorial_PY"] == "90"):
        return "No-Use"
    else:
        return "Editorial"

In [ ]:
# Applying the function to both dataframes and creating a new column with the names of the "Grupo_Editorial"
AllCDS["Grupo_Editorial_Nombre"] = AllCDS.apply(grupoeditorialnames, axis=1)
CSD2019["Grupo_Editorial_Nombre"] = CSD2019.apply(grupoeditorialnames, axis=1)

In [ ]:
AllCDS["Grupo_Editorial_Nombre"].value_counts()

In [ ]:
CSD2019["Grupo_Editorial_Nombre"].value_counts()

In [ ]:
# Defining the function to return 1 or 0 in the target variable according to the type of change
def targetvariable(df):
    if (df["Change"]) in {"No-Use_to_No-Use","SM_to_No-Use ","Editorial_to_No-Use", "New_course_No-Use"}:
        return 1
    else:
        return 0

In [ ]:
# Applying the function to create the target variable
AllCDS["Target"] = AllCDS.apply(targetvariable, axis=1)

In [ ]:
# Checking the change to no-use for 2017
AllCDS[AllCDS["Año_Natural"] == 2017]["Target"].value_counts()

In [ ]:
# Checking the change to no-use for 2018
AllCDS[AllCDS["Año_Natural"] == 2018]["Target"].value_counts()

In [ ]:
# Merging the dataframes to add information of the schools
AllData = pd.merge(AllCDS, clientes, on="Id_Cliente", how = "left")
AllCSD2019 = pd.merge(CSD2019, clientes, on="Id_Cliente", how = "left")

In [56]:
# Reordering columns
df = AllData[['Unique_Id','Id_Cliente', 'Año_Natural', 'Años_Curso','Curso','Asignatura','Tipo_Material_Educativo','Lengua','Tipo_Soporte_Actual', 'Variable_1','Variable_2','Latitud','Longitud', 'Comunidad_Autónoma', 'Id_Asociación', 'Id_Subasociación', 'Titularidad', 'Grupo_Editorial','Grupo_Editorial_PY', 'Grupo_Editorial_Nombre', 'Change','Target']]
AllCSD2019 = AllCSD2019[['Unique_Id','Id_Cliente', 'Año_Natural','Años_Curso','Curso','Asignatura','Tipo_Material_Educativo','Lengua','Tipo_Soporte_Actual', 'Variable_1','Variable_2','Latitud','Longitud', 'Comunidad_Autónoma', 'Id_Asociación', 'Id_Subasociación', 'Titularidad','Grupo_Editorial_PY', 'Grupo_Editorial_Nombre']]

In [54]:
AllData.to_csv(r'/Users/apkiener/Desktop/DataThon/Datathon.csv', header=True)




In [57]:
pd.pivot_table(df, values='Unique_Id', index=['Lengua'],
               columns=['Año_Natural'], aggfunc= "count", fill_value=0)

Año_Natural,2016,2017,2018
Lengua,,,
10,121336,119824,121451
13,397434,403070,405601
18,56030,55132,55484
26,37927,37604,37318
